[download inception-v3 model](http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz)

In [ ]:
#迁移学习主要代码

import glob
import os
import numpy as np
from tensorflow.python.platform import gfile
import tensorflow.contrib.slim as slim